In [1]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [2]:
import subprocess
from pathlib import Path
from collections import Counter

In [3]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if False:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8008/v1',
        '--api-key', 'xxx',
        '--model-name', 'Qwen/Qwen3-4B',
        '--chat-template-model-id', 'Qwen/Qwen3-4B',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

In [4]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

THREADS  = 60
MAX_PROMPTS = 60

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8008/v1',
    '--api-key', 'xxx',
    '--model-name', 'unsloth/gemma-3-4b-it',
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    #'--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--chunk-size', '20'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --model-name unsloth/gemma-3-4b-it --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 60 --max-prompts 60 --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --max-new-tokens 2000 --chunk-size 50
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   0%|          | 59/177477 [00:00<00:04, 38726.75prompt/s]
Preparing to process 60 new prompts in this run.
BANNED (prompt_idx=47, type=slop_phrase, phrase='static')   
BANNED (prompt_idx=5, type=slop_phrase, phrase='relentless')
BANNED (prompt_idx=22, type=slop_phrase, phrase='sky, casting long')     
BANNED (prompt_idx=6, type=ngram, ngram='felt like')                      
BANNED (prompt_idx=57, type=slop_phrase, phrase='lila')                   
BANNED (prompt_idx=20, type=slop_phrase, 

CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--model-name', 'unsloth/gemma-3-4b-it', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '60', '--max-prompts', '60', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--max-new-tokens', '2000', '--chunk-size', '50'], returncode=0)